In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional, Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from keras import backend as K
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

K.clear_session()

d:\PodPicks\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
# Load the dataset
file_path = '../Data/podcasts_data.csv'
dataset = pd.read_csv(file_path)
dataset.head()

,Genre,Podcast Name,Description,Publisher,Total Episodes,Spotify URL,Cover Image URL
0,arts and entertainment,Easy Stories in English,"Learning a language is hard, but Easy Stories ...","Ariel Goodbody, Polyglot English Teacher & Gla...",216,https://open.spotify.com/show/23zdIqNUb0riR51w...,https://i.scdn.co/image/ab6765630000ba8a767693...
1,arts and entertainment,Podcast Buku Kutu,"EPISODE BARU SETIAP SENIN, RABU, dan JUMAT -- ...",Aditya Hadi - PODLUCK,162,https://open.spotify.com/show/3w5zKrbQ6kgB0RKI...,https://i.scdn.co/image/ab6765630000ba8a04fa1a...
2,arts and entertainment,Underwood and Flinch and Other Audiobooks by M...,Underwood and Flinch is a three-time Parsec aw...,Mike Bennett,244,https://open.spotify.com/show/3VwIE3bG0zpTCNzR...,https://i.scdn.co/image/ab6765630000ba8a4e7b42...
3,arts and entertainment,Podcast Resensi Buku,Kumpulan resensi beragam buku berbagai genre d...,Podcast Resensi Buku - PODLUCK,264,https://open.spotify.com/show/6woLsDl6CSntzeWU...,https://i.scdn.co/image/ab6765630000ba8a1e97ef...
4,arts and entertainment,SupremeMasterTV,Supreme Master Television is an international ...,SupremeMasterTV,500,https://open.spotify.com/show/5bCgERRINgZWhauS...,https://i.scdn.co/image/ab6765630000ba8a7899e5...


In [5]:
# Cleaning
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = text.strip()  # Remove leading and trailing whitespace
    stop_words = set(stopwords.words('english', 'indonesian'))
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stop words
    return text

dataset['Podcast Name'] = dataset['Podcast Name'].apply(clean_text)
dataset['Genre'] = dataset['Genre'].apply(clean_text)

In [6]:
# Drop rows with NaN values in 'Podcast Name' column
podcast_data = dataset.dropna(subset=['Podcast Name'])

# Extract podcast names
podcast_names = podcast_data['Podcast Name'].values

# Extract relevant columns
podcast_names = podcast_data['Podcast Name'].values
podcast_genres = podcast_data['Genre'].values
podcast_descriptions = podcast_data['Description'].values
podcast_publishers = podcast_data['Publisher'].values
podcast_spotify_urls = podcast_data['Spotify URL'].values
podcast_cover_image_urls = podcast_data['Cover Image URL'].values

In [7]:
# Tokenization and Vectorization
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(podcast_names)

# Convert podcast names to sequences of integers
sequences = tokenizer.texts_to_sequences(podcast_names)

# Pad sequences to have the same length
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Get the vocabulary size for the embedding layer
vocab_size = len(tokenizer.word_index) + 1


In [8]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Dense(512, activation = 'relu'),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 128)           1153664   
                                                                 
 bidirectional (Bidirectiona  (None, 23, 256)          263168    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 23, 128)          164352    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 23, 128)           0         
                                                                 
 dense (Dense)               (None, 23, 512)           66048     
                                                                 
 time_distributed (TimeDistr  (None, 23, 9013)         4

In [9]:
# Prepare labels to match the output shape of the model
labels = np.expand_dims(padded_sequences, axis=-1)

# Train the model
model.fit(padded_sequences, labels, epochs=20, batch_size=128, validation_split=0.2)

Epoch 1/20
106/106 [==============================] - 7s 39ms/step - loss: 2.0096 - accuracy: 0.8619 - val_loss: 1.1202 - val_accuracy: 0.8722
Epoch 2/20
106/106 [==============================] - 3s 33ms/step - loss: 1.0422 - accuracy: 0.8709 - val_loss: 1.0660 - val_accuracy: 0.8771
Epoch 3/20
106/106 [==============================] - 3s 33ms/step - loss: 0.9855 - accuracy: 0.8762 - val_loss: 1.0266 - val_accuracy: 0.8793
Epoch 4/20
106/106 [==============================] - 3s 33ms/step - loss: 0.9169 - accuracy: 0.8789 - val_loss: 0.9634 - val_accuracy: 0.8815
Epoch 5/20
106/106 [==============================] - 4s 33ms/step - loss: 0.8565 - accuracy: 0.8810 - val_loss: 0.9431 - val_accuracy: 0.8840
Epoch 6/20
106/106 [==============================] - 3s 33ms/step - loss: 0.8099 - accuracy: 0.8828 - val_loss: 0.9235 - val_accuracy: 0.8857
Epoch 7/20
106/106 [==============================] - 3s 33ms/step - loss: 0.7550 - accuracy: 0.8856 - val_loss: 0.8858 - val_accuracy: 0.8903

In [10]:
def get_podcast_embeddings(model, data, batch_size=512):
    embeddings = []
    for i in range(0, len(data), batch_size):
        batch_data = data[i:i + batch_size]
        batch_embeddings = model.predict(batch_data)
        batch_embeddings = batch_embeddings.reshape(batch_embeddings.shape[0], -1)
        embeddings.append(batch_embeddings)
    embeddings = np.vstack(embeddings)
    return embeddings

# Prepare the embeddings for the podcasts
podcast_embeddings = get_podcast_embeddings(model, padded_sequences)

15/15 [==============================] - 1s 5ms/step


In [11]:
def search_podcasts(query, top_k=5):
    # Tokenize and pad the query
    query_sequence = tokenizer.texts_to_sequences([query])
    query_padded = pad_sequences(query_sequence, maxlen=max_length, padding='post')

    # Encode the query using the trained model
    query_embedding = model.predict(query_padded)
    query_embedding = query_embedding.reshape(1, -1)
    cosine_scores = cosine_similarity(query_embedding, podcast_embeddings)

    # Get the top_k similar podcasts
    top_k_indices = np.argsort(cosine_scores[0])[-top_k:][::-1]

    # Retrieve the corresponding podcast names
    similar_podcasts = [{
        'Name': podcast_names[idx],
        'Genre': podcast_genres[idx],
        #'Description': podcast_descriptions[idx],
        'Publisher': podcast_publishers[idx],
        'Spotify URL': podcast_spotify_urls[idx],
        'Cover Image URL': podcast_cover_image_urls[idx]
    } for idx in top_k_indices]

    return similar_podcasts

In [12]:
# Example search
query = "games"
similar_podcasts = search_podcasts(query)
print(similar_podcasts)

1/1 [==============================] - 0s 21ms/step
[{'Name': 'epiphantastic', 'Genre': 'health', 'Publisher': 'Shrestha S Bharadwaj', 'Spotify URL': 'https://open.spotify.com/show/6Jyd77b4lV8hQ2zqBdFEQf', 'Cover Image URL': 'https://i.scdn.co/image/ab6765630000ba8ac105f22d2145df57bb060798'}, {'Name': 'epiphantastic', 'Genre': 'lifestyle', 'Publisher': 'Shrestha S Bharadwaj', 'Spotify URL': 'https://open.spotify.com/show/6Jyd77b4lV8hQ2zqBdFEQf', 'Cover Image URL': 'https://i.scdn.co/image/ab6765630000ba8ac105f22d2145df57bb060798'}, {'Name': 'epiphantastic', 'Genre': 'books', 'Publisher': 'Shrestha S Bharadwaj', 'Spotify URL': 'https://open.spotify.com/show/6Jyd77b4lV8hQ2zqBdFEQf', 'Cover Image URL': 'https://i.scdn.co/image/ab6765630000ba8ac105f22d2145df57bb060798'}, {'Name': 'epiphantastic', 'Genre': 'selfcare', 'Publisher': 'Shrestha S Bharadwaj', 'Spotify URL': 'https://open.spotify.com/show/6Jyd77b4lV8hQ2zqBdFEQf', 'Cover Image URL': 'https://i.scdn.co/image/ab6765630000ba8ac105f22

In [13]:
model.save('model.h5')
print('Model saved succesfully')

Model saved succesfully


In [14]:
!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model "D:\PodPicks\Code\model.h5" "D:\PodPicks\Code\tfjs_model"